In [3]:
# netcdf/numpy/xray/stats
import xarray as xr

# plotting packages
import seaborn as sns
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap

# turbulence package
import turbpy
import turbpy.multiConst as mc

import sys
sys.path.append("/home/elilouis/sublimationofsnow/")
import sosutils
from metpy.units import units
import metpy
import pint_xarray
import altair as alt
alt.data_transformers.enable('json')
import pytz

from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import datetime as dt

# Inputs

In [4]:
tidy_df_30Min = pd.read_parquet("../sos/tidy_df_30Min_20221201_20230501_noplanar_fit.parquet")
tidy_df_30Min['time'] = pd.to_datetime(tidy_df_30Min['time'])

In [5]:
obukhov_src = tidy_df_30Min.query("measurement == 'Obukhov length'")

for tower in obukhov_src['tower'].unique():
    print(tower)
    for height in obukhov_src['height'].unique():
        print(height)
        src = obukhov_src.query(
                f"tower == '{tower}'"
            ).query(
                f"height == {height}"
            )
        tidy_df_30Min = sosutils.tidy_df_add_variable(
            tidy_df_30Min,
            src['height'] / src['value'],
            f"stability_function_{height}m_{tower}",
            "stability function",
            height,
            tower
        )

c
10.0
15.0
20.0
2.0
3.0
5.0


In [6]:
alt.Chart(tidy_df_30Min.query("measurement == 'stability function'")).mark_boxplot(outliers=False).encode(
    alt.X("height:O"),
    alt.Y("value:Q").title('stability function')
).properties(width = 200, height = 200, title='Without outliers') | \
alt.Chart(tidy_df_30Min.query("measurement == 'stability function'")).mark_boxplot(outliers={'size': 1}).encode(
    alt.X("height:O"),
    alt.Y("value:Q").title('stability function').scale(domain=[-100,100], clamp=True)
).properties(width = 200, height = 200, title='With outliers')

alt.HConcatChart(...)

In [9]:
alt.Chart(tidy_df_30Min.query("measurement == 'stability function'")).mark_line().encode(
    alt.X("time:T"),
    alt.Y("value:Q").scale(domain=[-100, 100], clamp=True),
    alt.Color("height:O").scale(scheme='viridis')
).properties(width = 800)

alt.Chart(...)

In [8]:
disdro_daily_max_precip_date_df = pd.read_csv("../sos/disdro_daily_max_precip_date_df.csv")

# Season Long

## Calculate turbulence statistics

In [ ]:
tke_vs_wind_speed_src = tidy_df_30Min[
    tidy_df_30Min.measurement.isin(['wind speed', 'turbulent kinetic energy', 'richardson number'])
].pivot_table(
    index=['time', 'tower', 'height'], 
    values='value', 
    columns='measurement'
).reset_index()

In [ ]:
tke_vs_wind_speed_src['Velocity scale'] = tke_vs_wind_speed_src['turbulent kinetic energy']**0.5
tke_vs_wind_speed_src['wind speed group'] = pd.cut(tke_vs_wind_speed_src['wind speed'], np.linspace(0, 12, 25, []))
tke_vs_wind_speed_src['wind speed group'] = tke_vs_wind_speed_src['wind speed group'].apply(lambda x: 0.5*(x.left + x.right))
tke_vs_wind_speed_src = tke_vs_wind_speed_src.query("tower == 'c'")

## Plot turbulence statistics for the winter season

In [ ]:
alt.Chart(
    tke_vs_wind_speed_src
).mark_bar().encode(
    alt.X(
        "wind speed:Q", 
        bin=alt.Bin(maxbins=50), 
    ),
    alt.Y("count():Q")
) | \
alt.Chart(
    tke_vs_wind_speed_src
).mark_bar().encode(
    alt.X("richardson number:Q", bin=alt.Bin(maxbins=50)),
    # alt.X("richardson number:Q", bin=alt.Bin(maxbins=50), scale=alt.Scale(domain=[-1,3], clamp=True), axis=alt.Axis(values=[-1, 0, 0.25, 1, 3], format='.02f')),
    alt.Y("count():Q")
)

In [ ]:
tke_stats = tke_vs_wind_speed_src.groupby(["height", "wind speed group"]).std()[['Velocity scale']].rename(
        columns={'Velocity scale': 'Vtke_std'}
    ).join(
        tke_vs_wind_speed_src.groupby(["height", "wind speed group"]).mean()[['Velocity scale']].rename(
            columns={'Velocity scale': 'Vtke_mean'}
        )
    ).reset_index()
tke_stats['lower'] = tke_stats.Vtke_mean - (1 * tke_stats.Vtke_std)
tke_stats['upper'] = tke_stats.Vtke_mean + (1 * tke_stats.Vtke_std)
tke_stats['wind speed group'] = tke_stats['wind speed group'].astype("float")

In [ ]:
bars = alt.Chart(tke_vs_wind_speed_src).mark_errorbar(extent='stdev').encode(
    alt.X("wind speed group:Q").title("Wind speed (m/s) (binned)"),
    alt.Y("Velocity scale:Q")
).properties(width=150, height = 100)

points = alt.Chart(tke_vs_wind_speed_src).mark_circle(color='black').encode(
    alt.X("wind speed group:Q"),
    alt.Y("mean(Velocity scale):Q")
)

(bars + points).facet('height:O', columns=3)

In [ ]:
alt.Chart(tke_vs_wind_speed_src).mark_line().encode(
    alt.X("wind speed group:Q"),
    alt.Y("mean(Velocity scale):Q")
).encode(alt.Color("height:O"))

In [ ]:
src = tke_vs_wind_speed_src.copy()
src = src[src.height.isin([2,10, 20])]
all_data_scaling_chart = alt.Chart(
    src
).mark_circle(opacity=0.15, size=5).encode(
    alt.X("wind speed:Q"),
    alt.Y("Velocity scale:Q").title('turbulence intensity')
).properties(width=150, height=100).facet(row='height:O')
all_data_scaling_chart

In [ ]:
tke_vs_wind_speed_src['threshold'] = tke_vs_wind_speed_src['wind speed'] > 3

In [ ]:
alt.Chart(
    tke_vs_wind_speed_src[tke_vs_wind_speed_src['richardson number'] > 0]
).transform_filter(
    alt.FieldLTPredicate("richardson number", 10)
).mark_circle(opacity=0.15, size=5).encode(
    alt.X("richardson number:Q", scale=alt.Scale(type='log', domain = [1e-3, 1e1], clamp=True)),
    alt.Y("Velocity scale:Q"),
    alt.Color("threshold:N")
).properties(width=175, height=175).facet('height:O')

In [ ]:
src = tke_vs_wind_speed_src.copy().set_index('time').loc['2023-02-19':'2023-02-25']
src = src[src['wind speed'] < 10]
src = src[src.height.isin([2,10, 20])]
all_data_scaling_chart = alt.Chart(
    src
).transform_filter(
    alt.FieldLTPredicate("wind speed", 10)
).mark_circle(opacity=0.25, size=8).encode(
    alt.X("wind speed:Q"),
    alt.Y("Velocity scale:Q", scale=alt.Scale(domain=[0,4], clamp=True), title='turbulence intensity')
).properties(width=150, height=100)
all_data_scaling_chart



In [ ]:
src = tke_vs_wind_speed_src.copy().set_index('time').loc['2023-03-16':'2023-03-20']
src = src[src['wind speed'] < 10]
src = src[src.height.isin([2,10, 20])]
all_data_scaling_chart = alt.Chart(
    src
).transform_filter(
    alt.FieldLTPredicate("wind speed", 10)
).mark_circle(opacity=0.25, size=8).encode(
    alt.X("wind speed:Q"),
    alt.Y("Velocity scale:Q", scale=alt.Scale(domain=[0,4], clamp=True), title='turbulence intensity')
).properties(width=150, height=100)
all_data_scaling_chart



In [ ]:
src = tke_vs_wind_speed_src.copy().set_index('time').loc['2023-01-31':'2023-02-05']
src = src[src['wind speed'] < 10]
src = src[src.height.isin([2,10, 20])]
all_data_scaling_chart = alt.Chart(
    src
).transform_filter(
    alt.FieldLTPredicate("wind speed", 10)
).mark_circle(opacity=0.25, size=8).encode(
    alt.X("wind speed:Q"),
    alt.Y("Velocity scale:Q", scale=alt.Scale(domain=[0,4], clamp=True), title='turbulence intensity')
).properties(width=150, height=100)
all_data_scaling_chart



In [ ]:
(
    alt.Chart(
        tke_vs_wind_speed_src
    ).transform_filter(
        alt.FieldLTPredicate("wind speed", 10)
    ).mark_circle(opacity=0.15).encode(
        alt.X("wind speed:Q"),
        alt.Y("Velocity scale:Q")
    ).properties(width=175, height=175).facet('height:O')
    &
    alt.Chart(
        tke_vs_wind_speed_src.set_index('time').loc['2022-12-17':'2022-12-21']
    ).transform_filter(
        alt.FieldLTPredicate("wind speed", 10)
    ).mark_circle(opacity=0.5).encode(
        alt.X("wind speed:Q"),
        alt.Y("Velocity scale:Q")
    ).properties(width=175, height=175).facet('height:O')
    &
    alt.Chart(
        tke_vs_wind_speed_src.set_index('time').loc['2023-02-19':'2023-02-25']
    ).transform_filter(
        alt.FieldLTPredicate("wind speed", 10)
    ).mark_circle(opacity=0.5).encode(
        alt.X("wind speed:Q"),
        alt.Y("Velocity scale:Q")
    ).properties(width=175, height=175).facet('height:O')
    &
    alt.Chart(
        tke_vs_wind_speed_src.set_index('time').loc['2023-01-31':'2023-02-04']
    ).transform_filter(
        alt.FieldLTPredicate("wind speed", 10)
    ).mark_circle(opacity=0.5).encode(
        alt.X("wind speed:Q"),
        alt.Y("Velocity scale:Q")
    ).properties(width=175, height=175).facet('height:O')
    &
    alt.Chart(
        tke_vs_wind_speed_src.set_index('time').loc['2023-02-04':'2023-02-08']
    ).transform_filter(
        alt.FieldLTPredicate("wind speed", 10)
    ).mark_circle(opacity=0.5).encode(
        alt.X("wind speed:Q"),
        alt.Y("Velocity scale:Q")
    ).properties(width=175, height=175).facet('height:O')
).resolve_scale(x='shared', y='shared')

In [ ]:
base = alt.Chart(
    tke_vs_wind_speed_src
).transform_filter(
    alt.FieldOneOfPredicate('height', [2,5,10,20])
).mark_line().encode(
    alt.X("wind speed group:Q"),
    alt.Y("mean(Velocity scale):Q"),
    alt.Color(
        "height:O", 
        # scale=alt.Scale(scheme='turbo')
    )
)

base.transform_filter(alt.FieldLTPredicate('richardson number', 0)).properties(title='Ri ∈ [-∞, 0)').properties(height=150) &\
base.transform_filter(alt.FieldGTPredicate('richardson number', 0)).transform_filter(alt.FieldLTPredicate('richardson number', 0.25)).properties(title='Ri ∈ [0, 0.25]').properties(height=150) &\
base.transform_filter(alt.FieldGTPredicate('richardson number', 0.25)).properties(title='Ri ∈ [0.25, ∞]').properties(height=150)

In [ ]:
base = alt.Chart(
    tke_vs_wind_speed_src
).transform_filter(
    alt.datum.tower == 'c'
).transform_filter(
    alt.FieldOneOfPredicate('height', [2, 10, 15, 20])
).mark_boxplot(size=10).encode(
    # alt.X("wind speed group:Q"),
    # alt.Y("Velocity scale:Q"),
    # alt.Color("height:O", scale=alt.Scale(scheme='turbo')),
    # alt.Facet("height:O")
    
    alt.X("height:O"),
    alt.Y("Velocity scale:Q"),
    alt.Color("height:O", scale=alt.Scale(scheme='turbo')),
    alt.Facet("wind speed group:Q", spacing=5, title='Wind Speed (binned)')
)

(
    base.transform_filter(alt.FieldLTPredicate('richardson number', 0)).properties(title='Ri ∈ [-∞, 0)').properties(height=150) &\
    base.transform_filter(alt.FieldGTPredicate('richardson number', 0)).transform_filter(alt.FieldLTPredicate('richardson number', 0.25)).properties(title='Ri ∈ [0, 0.25]').properties(height=150) &\
    base.transform_filter(alt.FieldGTPredicate('richardson number', 0.25)).properties(title='Ri ∈ [0.25, ∞]').properties(height=150)
).configure_view(
    strokeWidth=0
)

## Plot time series for winter season

In [ ]:
blowing_snow_chart = alt.Chart(tidy_df_30Min).transform_filter(
    alt.FieldOneOfPredicate('measurement', ['snow flux'])
).mark_line().encode(
    alt.X('time:T'),
    alt.Y("value:Q", title='Blowing Snow Flux (g/m^2)',
          scale=alt.Scale(domain=[0, 4], nice=False, clamp=True)
        ),
    alt.Color("height:N")
).properties(width=1400, height=200) 

days_since_precip_chart = alt.Chart(disdro_daily_max_precip_date_df).mark_line(color='black', strokeDash=[1,2]).encode(
    alt.X('time:T'),
    alt.Y("days_since_precip:Q"),
    alt.Color("variable:N")
).properties(width=1400, height=200)

alt.layer(blowing_snow_chart, days_since_precip_chart).resolve_scale(
    y = 'independent'
)

In [ ]:
(
    alt.Chart(tidy_df_30Min).transform_filter(
        alt.datum.measurement == 'w_h2o_'
    ).transform_filter(
        alt.datum.tower == 'ue'
    ).transform_window(
        rolling_mean = 'mean(value)',
        frame=[-12,12],
        groupby=['height','tower']
    ).mark_line().encode(
        alt.X('time:T'),
        alt.Y("rolling_mean:Q", title = 'Latent heat flux (g/m^2/s)'),
        alt.Color("height:O", scale=alt.Scale(scheme='viridis'))
    ).properties(width=1000, height=143).resolve_scale(color='independent') & \
    \
    alt.Chart(tidy_df_30Min).transform_filter(
        alt.FieldOneOfPredicate('measurement', ['cumulative sublimation (mm)'])
    ).mark_line().encode(
        alt.X('time:T'),
        alt.Y("value:Q", title='Cumulative sublimation (mm)'),
        alt.Color("height:N")
    ).properties(width=1000, height=143) & \
    \
    alt.Chart(tidy_df_30Min).transform_filter(
        alt.FieldOneOfPredicate('measurement', ['snow flux'])
    ).mark_line().encode(
        alt.X('time:T'),
        alt.Y("value:Q", title='Blowing Snow Flux (g/m^2)',
            scale=alt.Scale(domain=[0, 4], nice=False, clamp=True)
            ),
        alt.Color("height:N")
    ).properties(width=1000, height=71) & \
    \
    alt.Chart(disdro_daily_max_precip_date_df).mark_bar().encode(
        alt.X("time:T"),
        alt.Y("daily_precip (cm)", title='Daily precip (SWE, cm)')
    ).properties(width=1000, height=71) & \
    \
    alt.Chart(tidy_df_30Min).transform_filter(
        alt.FieldOneOfPredicate('measurement', ['richardson number'])
    ).transform_filter(
        alt.datum.tower == 'c'
    ).transform_filter(
        alt.datum.height == 2
    ).transform_window(
        rolling_mean = 'mean(value)',
        frame=[-12,12],
        groupby=['height','tower']
    ).mark_line().encode(
        alt.X('time:T'),
        alt.Y("rolling_mean:Q", title='Gradient Ri',
            scale=alt.Scale(domain=[0, 0.6], nice=False, clamp=True)
            )
    ).properties(width=1000, height=71) & \
    \
    alt.Chart(tidy_df_30Min).transform_filter(
        alt.datum.variable == 'Rsw_in_9m_d'
    ).mark_line().encode(
        alt.X('time:T'),
        alt.Y("value:Q", title = 'Incoming SW (w/m^2)')
    ).properties(width=1000, height=71)
).resolve_scale(color='independent', x='shared')

# Case Study

In [ ]:
case_study_start_date = 1
case_study_end_date = pd.datetime(2023,2,4)

## Plot time series

In [ ]:


surf_temp_chart = alt.Chart(src).transform_filter(
    alt.FieldOneOfPredicate('variable', ['Tsurf_c', 'Tsurf_d', 'Tsurf_ue', 'Tsurf_uw', 'Tsurf_rad_d'])
).transform_window(
    rolling_mean = 'mean(value)',
    frame = [-6, 6],
    groupby = ['variable']
).mark_line().encode(
    alt.X('time:T'),
    alt.Y("rolling_mean:Q", title='Temperature (˚C)'),
    alt.Color("variable:N")
).properties(width=800, height=143)


In [ ]:
case_study_start_date_1 = pd.datetime(2023,2,19)
case_study_end_date_1 = pd.datetime(2023,2,23)

case_study_start_date_2 = pd.datetime(2023,3,16)
case_study_end_date_2 = pd.datetime(2023,3,20)

case_study_start_date_3 = pd.datetime(2023,1,31)
case_study_end_date_3 = pd.datetime(2023,2,4)

case_study_1_src = tidy_df_30Min[tidy_df_30Min['time'] < case_study_end_date_1]
case_study_1_src = case_study_1_src[case_study_1_src['time'] >= case_study_start_date_1]
case_study_1_src = case_study_1_src.assign(case_study = 1)
case_study_1_src['time'] = case_study_1_src['time'] - dt.timedelta(days=49)

case_study_2_src = tidy_df_30Min[tidy_df_30Min['time'] < case_study_end_date_2]
case_study_2_src = case_study_2_src[case_study_2_src['time'] >= case_study_start_date_2]
case_study_2_src = case_study_2_src.assign(case_study = 2)
case_study_2_src['time'] = case_study_2_src['time'] - dt.timedelta(days=74)

case_study_3_src = tidy_df_30Min[tidy_df_30Min['time'] < case_study_end_date_3]
case_study_3_src = case_study_3_src[case_study_3_src['time'] >= case_study_start_date_3]
case_study_3_src = case_study_3_src.assign(case_study = 3)
case_study_3_src['time'] = case_study_3_src['time'] - dt.timedelta(days=30)

case_study_compare_src = pd.concat([case_study_1_src, case_study_2_src, case_study_3_src])

(
    # CUMULATIVE SUBLIMATION PLOT
    alt.Chart(case_study_compare_src).transform_filter(
        alt.FieldOneOfPredicate('measurement', ['cumulative sublimation (mm)'])
    ).transform_filter(
        alt.datum.height == 2
    ).mark_line().encode(
        alt.X('time:T', axis=alt.Axis(format="%d"), title='time (n days)'),
        alt.Y("value:Q", title='Cumulative sublimation (mm)', scale=alt.Scale(zero=False, nice=False)),
        alt.Color("height:N")
    ).properties(width=400, height=143).facet(column='case_study:O') &
    # LATENT HEAT FLUX PLOT
    alt.Chart(case_study_compare_src).transform_filter(
        alt.datum.variable == "w_h2o__2m_c"
    ).transform_filter(
        alt.datum.tower == 'c'
    ).transform_window(
        rolling_mean = 'mean(value)',
        frame= [-6, 6],
        groupby = ["height"]
    ).mark_line().encode(
        alt.X('time:T', axis=alt.Axis(format="%d"), title='time (n days)'),
        alt.Y("rolling_mean:Q", title = 'Latent heat flux (g/m^2/s)'),
        alt.Color("height:N")
    ).properties(width=400, height=143).facet(column='case_study:O') & 
    # Richardson number plot
    alt.Chart(case_study_compare_src).transform_filter(
        alt.FieldOneOfPredicate('variable', ['Ri_2m_c_Tsurf_rad_d'])
    ).mark_line().encode(
        alt.X('time:T', axis=alt.Axis(format="%d"), title='time (n days)'),
        alt.Y("value:Q", title='Ri'),
    ).properties(width=400, height=143).facet(column='case_study:O') &
    # Surface and Air Temp Plot
    alt.Chart(case_study_compare_src).transform_filter(
        alt.FieldOneOfPredicate('variable', ['T_2m_c', 'Tsurf_rad_d'])
    ).mark_line().encode(
        alt.X('time:T', axis=alt.Axis(format="%d"), title='time (n days)'),
        alt.Y("value:Q", title="Temperature (˚C)"),
        alt.StrokeDash("measurement:N")
    ).properties(width=400, height=143).facet(column='case_study:O')
)

In [ ]:
src = tidy_df_30Min[tidy_df_30Min['time'] < case_study_end_date]
src = src[src['time'] >= case_study_start_date]
wind_speed_chart = alt.Chart(src).transform_filter(
    alt.FieldOneOfPredicate('variable', ['spd_2m_c', 'spd_10m_c', 'spd_20m_c'])
).mark_line().encode(
    alt.X('time:T'),
    alt.Y("value:Q", title='Wind Speed (m/s)'),
    alt.Color("height:N")
).properties(width=800, height=71)

tke_chart = alt.Chart(src).transform_filter(
    alt.FieldOneOfPredicate('variable', ['tke_2m_c', 'tke_10m_c', 'tke_20m_c'])
).mark_line().encode(
    alt.X('time:T'),
    alt.Y("value:Q", title='TKE', scale=alt.Scale(type='log')),
    alt.Color("height:N")
).properties(width=800, height=71)

ri_plot = alt.Chart(src).transform_filter(
    alt.FieldOneOfPredicate('variable', ['Ri_2m_c', 'Ri_2m_c_Tsurf_rad_d'])
).mark_line().encode(
    alt.X('time:T'),
    alt.Y("value:Q", title='Ri', scale=alt.Scale()),
    alt.Color("variable:N")
).properties(width=800, height=71)

surf_temp_chart = alt.Chart(src).transform_filter(
    alt.FieldOneOfPredicate('variable', ['Tsurf_c', 'Tsurf_d', 'Tsurf_ue', 'Tsurf_uw', 'Tsurf_rad_d'])
).transform_window(
    rolling_mean = 'mean(value)',
    frame = [-6, 6],
    groupby = ['variable']
).mark_line().encode(
    alt.X('time:T'),
    alt.Y("rolling_mean:Q", title='Temperature (˚C)'),
    alt.Color("variable:N")
).properties(width=800, height=143)

air_temp_chart = alt.Chart(src).transform_filter(
    alt.FieldOneOfPredicate('variable', ['T_2m_c'])
).transform_window(
    rolling_mean = 'mean(value)',
    frame = [-6, 6],
    groupby = ['variable']
).mark_line(color='black', strokeDash=[5,5]).encode(
    alt.X('time:T'),
    alt.Y("rolling_mean:Q", title='Temperature (˚C)'),
).properties(width=800, height=143)

measured_lh_flux_plot = alt.Chart(src).transform_filter(
    alt.datum.variable == 'w_h2o__2m_c'
).mark_point(color='black', size=10).encode(
    alt.X("time:T"),
    alt.Y("value:Q", title = 'Latent heat flux (g/m^2/s)', scale=alt.Scale())
).properties(width=800, height=143)

predicted_lh_flux_plot = alt.Chart(src).transform_filter(
    alt.FieldOneOfPredicate('variable', ['w_h20_modeled_MO_HdB_2m_c', 'w_h20_modeled_MO_HdB_2m_c_Tsurf_rad_d'])
).mark_line().encode(
    alt.X('time:T'),
    alt.Y("value:Q"),
    alt.Color("measurement:N")
).properties(width=800, height=143)

snow_flux_chart = alt.Chart(src).transform_filter(
    alt.FieldOneOfPredicate('measurement', ['snow flux'])
).mark_line().encode(
    alt.X('time:T'),
    alt.Y("value:Q", title='Blowing Snow Flux (g/m^2)'),
    alt.Color("height:N")
).properties(width=800, height=71)

(wind_speed_chart & tke_chart & ri_plot & (surf_temp_chart + air_temp_chart) & (predicted_lh_flux_plot + measured_lh_flux_plot) & snow_flux_chart).resolve_scale(
    color='independent'
).configure_legend(
    labelLimit = 0
)

# Plot M.O.S.T model results

In [ ]:
wide_df = tidy_df_30Min.pivot_table(
    index=['time'],
    columns="variable",
    values='value'
)[[
    'w_h2o__2m_c',

    'w_h20_modeled_Standard_2m_c',
    'w_h20_modeled_MO_HdB_2m_c',
    
    'Ri_2m_c',
    'SF_avg_1m_ue',
    'SF_avg_2m_ue',

    'spd_2m_c',
    'dir_2m_c',
    'u_2m_c',
    'v_2m_c',
    'w_2m_c',
    'u_w__2m_c',
    'v_w__2m_c',
    'u_tc__2m_c',
    'v_tc__2m_c',
    'u_h2o__2m_c',
    'v_h2o__2m_c',
    'w_tc__2m_c',
    'tc_2m_c',
    'T_2m_c', 
    'RH_2m_c'
]]

In [ ]:
wide_df['Holtslag/de Bruin residual'] = wide_df['w_h20_modeled_MO_HdB_2m_c'] - wide_df['w_h2o__2m_c']
wide_df['Standard residual'] =  wide_df['w_h20_modeled_Standard_2m_c'] - wide_df['w_h2o__2m_c']

wide_df['Holtslag/de Bruin residual normalized'] = wide_df['Holtslag/de Bruin residual'] / wide_df['w_h2o__2m_c']
wide_df['Standard residual normalized'] = wide_df['Standard residual'] / wide_df['w_h2o__2m_c']

In [ ]:
wide_df.to_csv("monin_obukhov_with_turbpy_seasonlong.csv")

In [ ]:
wide_df = pd.read_csv("monin_obukhov_with_turbpy_seasonlong.csv")

In [ ]:
fig, axes = plt.subplots(1,2, figsize =(10,5))

axes[0].scatter(
    wide_df['w_h2o__2m_c'],
    wide_df['w_h20_modeled_Standard_2m_c'],
    s=1,
    alpha=0.1
)
axes[0].set_title("All Data\nStandard bulk aerodynamic method")

axes[1].scatter(
    wide_df['w_h2o__2m_c'],
    wide_df['w_h20_modeled_Standard_2m_c'],
    s=1,
    alpha=0.1
)
axes[1].set_title("All Data\nHoltslag and de Bruin method for snow and ice")

for ax in axes:
    ax.set_xlim(-0.005, 0.025)
    ax.set_ylim(-0.005, 0.025)
    ax.set_xlabel("Measured Vertical Moisture Flux (g/m²/s)")
    ax.set_ylabel("Predicted Vertical Moisture Flux (g/m²/s)")
    # ax.set_aspect("equal")

plt.tight_layout()

# Examine multiple explanatory variables

In [ ]:
fig, axes = plt.subplots(2,5, figsize =(30,10))

sns.scatterplot(x=wide_df['Ri_2m_c'],       y=wide_df['Holtslag/de Bruin residual'], s=15, alpha=0.10, ax=axes[0,0])
sns.scatterplot(x=wide_df['SF_avg_1m_ue'],  y=wide_df['Holtslag/de Bruin residual'], s=50, alpha=0.10, ax=axes[0,1])
sns.scatterplot(x=wide_df['SF_avg_2m_ue'],  y=wide_df['Holtslag/de Bruin residual'], s=50, alpha=0.10, ax=axes[0,2])
sns.scatterplot(x=wide_df['spd_2m_c'],      y=wide_df['Holtslag/de Bruin residual'], s=15, alpha=0.10, ax=axes[0,3])
sns.scatterplot(x=wide_df['dir_2m_c'],      y=wide_df['Holtslag/de Bruin residual'], s=15, alpha=0.10, ax=axes[0,4])

sns.scatterplot(x=wide_df['w_2m_c'],        y=wide_df['Holtslag/de Bruin residual'], s=15, alpha=0.10, ax=axes[1,0])
sns.scatterplot(x=wide_df['w_tc__2m_c'],    y=wide_df['Holtslag/de Bruin residual'], s=15, alpha=0.10, ax=axes[1,1])
sns.scatterplot(x=wide_df['tc_2m_c'],       y=wide_df['Holtslag/de Bruin residual'], s=15, alpha=0.10, ax=axes[1,2])
sns.scatterplot(x=wide_df['T_2m_c'],        y=wide_df['Holtslag/de Bruin residual'], s=15, alpha=0.10, ax=axes[1,3])
sns.scatterplot(x=wide_df['RH_2m_c'],       y=wide_df['Holtslag/de Bruin residual'], s=15, alpha=0.10, ax=axes[1,4])

for ax in axes.flatten():
    ax.set_ylim(-0.01, 0.01)
    ax.axhline(y = 0.0, color = 'k')

# axes[0,0].set_xlim(-0.5, 0.10)
# axes[0,1].set_xlim(0, 0.25)
# axes[0,2].set_xlim(0, 0.25)
# axes[0,3].set_xlim(0, 10)
# axes[0,4].set_xlim(0, 360)
# axes[1,1].set_xlim(-0.2, 0.1)

# sns.scatterplot(wide_df['u_2m_c'], wide_df['Holtslag/de Bruin residual'], s=1, alpha=0.1, ax=axes[0])
# sns.scatterplot(wide_df['v_2m_c'], wide_df['Holtslag/de Bruin residual'], s=1, alpha=0.1, ax=axes[0])
# sns.scatterplot(wide_df['u_w__2m_c'], wide_df['Holtslag/de Bruin residual'], s=1, alpha=0.1, ax=axes[0])
# sns.scatterplot(wide_df['v_w__2m_c'], wide_df['Holtslag/de Bruin residual'], s=1, alpha=0.1, ax=axes[0])
# sns.scatterplot(wide_df['u_tc__2m_c'], wide_df['Holtslag/de Bruin residual'], s=1, alpha=0.1, ax=axes[0])
# sns.scatterplot(wide_df['v_tc__2m_c'], wide_df['Holtslag/de Bruin residual'], s=1, alpha=0.1, ax=axes[0])
# sns.scatterplot(wide_df['u_h2o__2m_c'], wide_df['Holtslag/de Bruin residual'], s=1, alpha=0.1, ax=axes[0])
# sns.scatterplot(wide_df['v_h2o__2m_c'], wide_df['Holtslag/de Bruin residual'], s=1, alpha=0.1, ax=axes[0])

plt.tight_layout()

# Exclude outliers for richardson number residual comparison

In [ ]:
ri_IQR = np.percentile(wide_df['Ri_2m_c'], 75) - np.percentile(wide_df['Ri_2m_c'], 25)
ri_max = np.percentile(wide_df['Ri_2m_c'], 75) + 1.5*ri_IQR
ri_min = np.percentile(wide_df['Ri_2m_c'], 75) - 1.5*ri_IQR

In [ ]:
len(wide_df), len(wide_df.query(f"Ri_2m_c > {ri_min}").query(f"Ri_2m_c < {ri_max}"))

In [ ]:
residual_IQR = np.nanpercentile(wide_df['Holtslag/de Bruin residual'], 75) - np.nanpercentile(wide_df['Holtslag/de Bruin residual'], 25)
residual_max = np.nanpercentile(wide_df['Holtslag/de Bruin residual'], 75) + 1.5*residual_IQR
residual_min = np.nanpercentile(wide_df['Holtslag/de Bruin residual'], 75) - 1.5*residual_IQR

In [ ]:
len(wide_df), len(wide_df[(wide_df["Holtslag/de Bruin residual"] > residual_min) & (wide_df["Holtslag/de Bruin residual"] < residual_max)])

In [ ]:
src = wide_df.query(f"Ri_2m_c > {ri_min}").query(f"Ri_2m_c < {ri_max}")
src = src[(src["Holtslag/de Bruin residual"] > residual_min) & (src["Holtslag/de Bruin residual"] < residual_max)]

g = sns.JointGrid(data = src, x='Ri_2m_c',       y='Holtslag/de Bruin residual', 
                #   cbar=True, binwidth=(0.01, 0.00025)
                )
g.plot_joint(sns.histplot)
g.plot_marginals(sns.kdeplot)
# plt.gca().set_xlim(-0.5, 0.10)
# plt.gca().set_ylim(-0.01, 0.01)